In [1]:
import dualcodec
import torchaudio
from IPython.display import Audio

/home/vansh/dualcodec-exp/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_id = "12hz_v1"

In [3]:
path_1 = "output_checkpoints_3/dualcodec_experiments_causal_decoder/checkpoint/epoch-0000_step-0052000_loss-210.759888-dualcodec_experiments_causal_decoder"

In [4]:
dualcodec_model = dualcodec.get_model(model_id, pretrained_model_path=path_1, is_checkpoint=True) 

/home/vansh/dualcodec-exp/.venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Loading model from output_checkpoints_3/dualcodec_experiments_causal_decoder/checkpoint/epoch-0000_step-0052000_loss-210.759888-dualcodec_experiments_causal_decoder/model.safetensors
Model loaded


In [5]:
dualcodec_inference = dualcodec.InferenceWhisper(dualcodec_model=dualcodec_model)

Fetching 7 files: 100%|██████████| 7/7 [00:00<00:00, 63412.80it/s]

In [6]:
dualcodec_inference.semantic_cfg.semantic_model.encoder.causal_mask.shape

torch.Size([1, 1, 1500, 1500])

In [7]:
audio, sr = torchaudio.load("audio_samples/tara.wav")
audio = torchaudio.functional.resample(audio, sr, 24000)
audio = audio.reshape(1,1,-1)
audio = audio.cpu()

Audio(audio.squeeze(0), rate=24000)

In [8]:
semantic_codes, acoustic_codes = dualcodec_inference.encode(audio, n_quantizers=8)

audio_16k.shape torch.Size([1, 1, 480000])
audio_input.shape torch.Size([480000])
input_features.shape torch.Size([80, 3000])


`sdpa` attention does not support `output_attentions=True` or `head_mask`. Please set your attention to `eager` if you want any of these features.


feat.shape torch.Size([1, 1024, 1500])
feat.shape after avg_pool1d torch.Size([1, 1024, 375])


In [9]:
print(semantic_codes.shape)
print(acoustic_codes.shape)

torch.Size([1, 1, 375])
torch.Size([1, 7, 375])


In [10]:
out_audio = dualcodec_inference.decode(semantic_codes, acoustic_codes)
Audio(out_audio.cpu().squeeze(0), rate=24000)

In [11]:
dualcodec_inference_normal = dualcodec.Inference(dualcodec_model=dualcodec_model)

Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 65536.00it/s]

In [12]:
semantic_codes_2, acoustic_codes_2 = dualcodec_inference_normal.encode(audio, n_quantizers=8)

audio_16k.shape: torch.Size([1, 1, 480000])
inputs: {'input_features': tensor([[[ 3.4444,  4.1505,  3.5429,  ..., -0.8546, -0.9828, -0.9032],
         [ 3.1201,  3.7248,  3.6944,  ..., -0.8725, -0.7046, -1.0085],
         [ 2.2194,  3.1248,  2.6875,  ..., -0.7682, -0.9017, -0.9863],
         ...,
         [ 0.3568,  0.9021,  0.4771,  ..., -0.7608, -0.9619, -0.7482],
         [-0.8716, -0.4984, -1.0532,  ..., -0.5468, -0.7563, -0.6447],
         [-0.4128, -0.0291, -1.0439,  ..., -0.9225, -0.8868, -0.7765]]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], dtype=torch.int32)}
input_features.shape: torch.Size([1499, 160])
attention_mask.shape: torch.Size([1499])
input_features.shape: torch.Size([1, 1499, 160])
feat.shape: torch.Size([1, 1024, 1499])
feat.shape after avg_pool1d: torch.Size([1, 1024, 374])
